In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import espn_scraper as espn #https://github.com/andr3w321/espn_scraper

def strzero(num):
    num = str(num)
    if len(num) == 1:
        num = '0'+num
    return(num)

In [3]:
year = 2018
month = 9
day = 12
date = '20190822'
schedURL = 'https://www.espn.com/wnba/scoreboard/_/date/'+str(year)+strzero(month)+strzero(day)

In [4]:
game_ids = []
game_info = []
years = list(range(1997,2022))
years = [2021]
years.reverse()

In [5]:
years

[2021]

In [6]:
#get all of the game ids ever
for year in years:
    prev_len = len(game_ids)
    scoreboard_urls= espn.get_all_scoreboard_urls('wnba', year) 
    for scoreboard_url in scoreboard_urls:
        data = espn.get_url(scoreboard_url, cached_path="cached_data")
        try:
            for event in data['content']:  
                if event['analytics']['gameId'] not in game_ids:
                    game_ids.append(event['analytics']['gameId'])
                    game_info.append([event['analytics']['gameId'],year])
        except:
            pass
    print([year,len(game_ids)-prev_len])


[2021, 218]


In [7]:
game_id_df = pd.DataFrame(game_info)
game_id_df.columns = ['GameID','year']
game_id_df.to_csv('game_info.csv')

In [8]:
len(game_ids)

218

In [32]:
if True:
#def get_game_data(GameID):
    #first get all the game info
    data_type = "recap"
    url = espn.get_game_url(data_type, "wnba", GameID)
    data = espn.get_url(url)
    temp = data['gamepackageJSON']
    temp = temp['header']   
    #date
    Date = temp['competitions'][0]['date']
    #winning team
    if temp['competitions'][0]['competitors'][0]['winner']: #check if 1st team won
        WinningTeam = temp['competitions'][0]['competitors'][0]['team']['abbreviation']
    else: #otehrwise 2nd team won
        WinningTeam = temp['competitions'][0]['competitors'][1]['team']['abbreviation']
    #regular season or playoffs?
    gametypes = ['none','preseason','regular season','playoffs']
    gametype_int = temp['season']['type']
    GameType = gametypes[gametype_int]
    
    ########################################
    #now get all the play-by-play
    GameID = str(GameID)
    URL = 'https://www.espn.com/wnba/playbyplay?gameId='+GameID
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #teams
    teams = [x.text for x in soup.find_all('span', class_='abbrev')]
    AwayTeam = teams[0]
    HomeTeam = teams[1]
    teams = {teams[0]:'away',teams[1]:'home'}
    
    #scrape the shot description table
    times = [x.text for x in soup.find_all('td', class_='time-stamp')]
    plays = [x.text for x in soup.find_all('td', class_='game-details')]
    score = [x.text for x in soup.find_all('td', class_='combined-score')]
    
    #get team names from the logo image urls
    logourl = [x.find('img').get('src') for x in soup.find_all('td', class_='logo')]
    team = [re.match(r"https.*/wnba/500/(?P<team>[a-z]+)\.",x).group('team').upper() for x in logourl]
    homeoraway = [teams[x] for x in team]
    
    #convert to df
    shotdescriptions = pd.DataFrame({'Time': times,
                          'EventDescription': plays,
                          'Score': score,
                          'EventTeam': team,
                          'HomeOrAway':homeoraway})
    shotdescriptions['GameID'] = GameID
    shotdescriptions['Date'] = Date
    shotdescriptions['GameType'] = GameType
    shotdescriptions['WinningTeam'] = WinningTeam
    shotdescriptions['AwayTeam'] = AwayTeam
    shotdescriptions['HomeTeam'] = HomeTeam
    
    
    #get shot chart data for away team
    awayshots = soup.find('ul', class_='shots away-team')
    awayshots = awayshots.find_all('li')
    awayshotchart = []
    for row in awayshots:
        coordtext = row.get('style')
        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
        xcoord = match.group('xcoord')
        ycoord = match.group('ycoord')
        ycoord = str(100-float(ycoord))
        row_data = [row.get('class')[0],
                    row.get('data-homeaway'),
                    row.get('data-period'),
                    row.get('data-shooter'),
                    row.get('data-text'),
                    row.get('id'),
                    xcoord,
                    ycoord]
        awayshotchart.append(row_data)
    
    #get shot chart data for home team
    homeshots = soup.find('ul', class_='shots home-team')
    homeshots = homeshots.find_all('li')
    homeshotchart = []
    for row in homeshots:
        coordtext = row.get('style')
        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
        xcoord = match.group('xcoord')
        ycoord = match.group('ycoord')
        ycoord = str(100-float(ycoord))
        row_data = [row.get('class')[0],
                    row.get('data-homeaway'),
                    row.get('data-period'),
                    row.get('data-shooter'),
                    row.get('data-text'),
                    row.get('id'),
                    xcoord,
                    ycoord]
        homeshotchart.append(row_data)
    
    awayshotchart = pd.DataFrame(awayshotchart)
    homeshotchart = pd.DataFrame(homeshotchart)
    
    #name columns for shot chart data
    awayshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
    homeshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
    
    #join to shot descriptions
    playindex = 0
    for row in range(len(awayshotchart)):
        not_found = True
        while not_found:
            if shotdescriptions.loc[playindex,'EventDescription']==awayshotchart.loc[row,'EventDescription']:
                if shotdescriptions.loc[playindex,'HomeOrAway']=='away':
                    shotdescriptions.loc[playindex,'ShotOutcome']=awayshotchart.loc[row,'ShotOutcome']
                    shotdescriptions.loc[playindex,'Quarter']=awayshotchart.loc[row,'Quarter']
                    shotdescriptions.loc[playindex,'pid']=awayshotchart.loc[row,'pid']
                    shotdescriptions.loc[playindex,'ShotID']=awayshotchart.loc[row,'ShotID']
                    shotdescriptions.loc[playindex,'xcoord']=awayshotchart.loc[row,'xcoord']
                    shotdescriptions.loc[playindex,'ycoord']=awayshotchart.loc[row,'ycoord']
                    not_found = False
            playindex += 1       
    
    playindex = 0
    for row in range(len(homeshotchart)):
        not_found = True
        while not_found:
            if shotdescriptions.loc[playindex,'EventDescription']==homeshotchart.loc[row,'EventDescription']:
                if shotdescriptions.loc[playindex,'HomeOrAway']=='home':
                    shotdescriptions.loc[playindex,'ShotOutcome']=homeshotchart.loc[row,'ShotOutcome']
                    shotdescriptions.loc[playindex,'Quarter']=homeshotchart.loc[row,'Quarter']
                    shotdescriptions.loc[playindex,'pid']=homeshotchart.loc[row,'pid']
                    shotdescriptions.loc[playindex,'ShotID']=homeshotchart.loc[row,'ShotID']
                    shotdescriptions.loc[playindex,'xcoord']=homeshotchart.loc[row,'xcoord']
                    shotdescriptions.loc[playindex,'ycoord']=homeshotchart.loc[row,'ycoord']
                    not_found = False
            playindex += 1      
    
    #get quarter numbers and scores
    quarter = 1
    shotdescriptions.loc[0,'AwayScore'] = 0
    shotdescriptions.loc[0,'HomeScore'] = 0
    for row in range(0,len(shotdescriptions)):
        if re.match('End of Game',shotdescriptions.loc[row,'EventDescription']) :
            quarter -= 1 #make the end of the game the same as the last quarter
        shotdescriptions.loc[row,'Quarter'] = quarter
        match = re.match(("([0-9]+) - ([0-9]+)"),shotdescriptions.loc[row,'Score'])
        shotdescriptions.loc[row,'AwayScore'] = match.group(1)
        shotdescriptions.loc[row,'HomeScore'] = match.group(2)
        if re.match('End',shotdescriptions.loc[row,'EventDescription']) :
            quarter += 1 #if you reach the end of the quarter, go to the next quarter
    
    #return shotdescriptions

IndexError: list index out of range

In [23]:
espn.get_game_url('playbyplay', "wnba", 401244185)
data = espn.get_url(url)

https://www.espn.com/wnba/recap?gameId=401370398&xhr=1


JSONDecodeError: [Errno Expecting value] 
        <!doctype html>
        <html lang="en">
            <head>
                <meta charSet="utf-8" />

                <!-- ESPNFITT | e8150d962168 | 6039 | 289220859142bfaf08da10832258c3f071e72acf | Sat, 30 Jul 2022 04:03:06 GMT -->
                <script type="text/javascript" src="https://dcf.espn.com/TWDC-DTCI/prod/Bootstrap.js" async></script>
                <script type="text/javascript">
        ;(function(){
            function gc(n){n=document.cookie.match("(^|;) ?"+n+"=([^;]*)(;|$)");return n?n[2]:null}function sc(n){document.cookie=n}function smpl(n){n/=100;return!!n&&Math.random()<=n}var _nr=!1,_nrCookie=gc("_nr");null!==_nrCookie?"1"===_nrCookie&&(_nr=!0):smpl(100)?(_nr=!0,sc("_nr=1; path=/")):(_nr=!1,sc("_nr=0; path=/"));;
            _nr && window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true}};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0]

In [11]:
#game_ids_csv = pd.read_csv('not-scraped.csv')

In [12]:
df = pd.DataFrame()
#game_ids_test = [401230835]
game_ids_test = game_ids
for GameID in game_ids_test:
    try:
        temp = get_game_data(GameID)
        df = df.append(temp)
    except:
        print(GameID)

https://www.espn.com/wnba/recap?gameId=401322870&xhr=1
401322870
https://www.espn.com/wnba/recap?gameId=401323518&xhr=1
401323518
https://www.espn.com/wnba/recap?gameId=401322871&xhr=1
401322871
https://www.espn.com/wnba/recap?gameId=401323519&xhr=1
401323519
https://www.espn.com/wnba/recap?gameId=401323520&xhr=1
401323520
https://www.espn.com/wnba/recap?gameId=401323454&xhr=1
401323454
https://www.espn.com/wnba/recap?gameId=401323455&xhr=1
401323455
https://www.espn.com/wnba/recap?gameId=401320565&xhr=1
401320565
https://www.espn.com/wnba/recap?gameId=401320566&xhr=1
401320566
https://www.espn.com/wnba/recap?gameId=401320567&xhr=1
401320567
https://www.espn.com/wnba/recap?gameId=401320568&xhr=1
401320568
https://www.espn.com/wnba/recap?gameId=401320569&xhr=1
401320569
https://www.espn.com/wnba/recap?gameId=401320570&xhr=1
401320570
https://www.espn.com/wnba/recap?gameId=401320571&xhr=1
401320571
https://www.espn.com/wnba/recap?gameId=401320572&xhr=1
401320572
https://www.espn.com/wnba

401353913
https://www.espn.com/wnba/recap?gameId=401320684&xhr=1
401320684
https://www.espn.com/wnba/recap?gameId=401320685&xhr=1
401320685
https://www.espn.com/wnba/recap?gameId=401320686&xhr=1
401320686
https://www.espn.com/wnba/recap?gameId=401320687&xhr=1
401320687
https://www.espn.com/wnba/recap?gameId=401320688&xhr=1
401320688
https://www.espn.com/wnba/recap?gameId=401320689&xhr=1
401320689
https://www.espn.com/wnba/recap?gameId=401320690&xhr=1
401320690
https://www.espn.com/wnba/recap?gameId=401320691&xhr=1
401320691
https://www.espn.com/wnba/recap?gameId=401320692&xhr=1
401320692
https://www.espn.com/wnba/recap?gameId=401320693&xhr=1
401320693
https://www.espn.com/wnba/recap?gameId=401320694&xhr=1
401320694
https://www.espn.com/wnba/recap?gameId=401320695&xhr=1
401320695
https://www.espn.com/wnba/recap?gameId=401320696&xhr=1
401320696
https://www.espn.com/wnba/recap?gameId=401320697&xhr=1
401320697
https://www.espn.com/wnba/recap?gameId=401320698&xhr=1
401320698
https://www.esp

In [13]:
df.to_csv('data/raw-data/espn-raw-pbp-2021.csv',index=False)

In [14]:
game_ids[0:3]

[401322870, 401323518, 401322871]

In [15]:
GameID = str(GameID)
URL = 'https://www.espn.com/wnba/boxscore?gameId='+GameID
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

soup.find_all('span', class_='team-name')

[]

In [49]:
get_game_data(401105046)

https://www.espn.com/wnba/recap?gameId=401105046&xhr=1


JSONDecodeError: [Errno Expecting value] 
        <!doctype html>
        <html lang="en">
            <head>
                <meta charSet="utf-8" />

                <!-- ESPNFITT | 3bdb2d32cd46 | 6039 | 289220859142bfaf08da10832258c3f071e72acf | Sat, 30 Jul 2022 04:42:50 GMT -->
                <script type="text/javascript" src="https://dcf.espn.com/TWDC-DTCI/prod/Bootstrap.js" async></script>
                <script type="text/javascript">
        ;(function(){
            function gc(n){n=document.cookie.match("(^|;) ?"+n+"=([^;]*)(;|$)");return n?n[2]:null}function sc(n){document.cookie=n}function smpl(n){n/=100;return!!n&&Math.random()<=n}var _nr=!1,_nrCookie=gc("_nr");null!==_nrCookie?"1"===_nrCookie&&(_nr=!0):smpl(100)?(_nr=!0,sc("_nr=1; path=/")):(_nr=!1,sc("_nr=0; path=/"));;
            _nr && window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true}};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0]

In [24]:
URL = "https://www.espn.com/wnba/playbyplay/_/gameId/401370398"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

soup.find_all('span', class_='team-name')

[]

In [30]:
with open('example-game.txt', 'w') as f:
    f.write(str(soup))